In [ ]:
from typing import List, Tuple
import cv2 as cv
import cv2
import numpy as np
from PIL import Image
from IPython.display import display
from contextlib import contextmanager
import contextlib
import time
import sys
import enum
from enum import Enum
import itertools

def erode(img: np.ndarray, sz: int, shape):
    el = cv.getStructuringElement(shape, (2 * sz + 1, 2 * sz + 1), (sz, sz))
    v = cv.erode(img, el)
    return v

def dilate(img: np.ndarray, sz: int, shape):
    el = cv.getStructuringElement(shape, (2 * sz + 1, 2 * sz + 1), (sz, sz))
    v = cv.dilate(img, el)
    return v

#del out, outv, outh

class timer_unit(Enum):
    SECOND = 1
    MILLISECOND = 2

@contextlib.contextmanager
def timer(unit: timer_unit = timer_unit.SECOND):
    if unit is timer_unit.SECOND:
        t0 = time.time()
        yield lambda : time.time() - t0
    elif unit is timer_unit.MILLISECOND:
        t0 = int(1000*time.time())
        yield lambda : int(1000*time.time()) - t0

@contextlib.contextmanager
def timer_sec():
    t0 = time.time()
    yield lambda : time.time() - t0

@contextlib.contextmanager
def timer_ms():
    t0 = int(1000*time.time())
    yield lambda : int(1000*time.time()) - t0



im = cv.imread('../tmp/test_get_non_ui_screen_non_ui_img.bmp')

def get_grid(im, threshold = 100, grid_color = (0,0,0)):
    def get_trilples(lst):
        triples = []
        if len(lst) >= 3:
            for i in range(len(lst)-2):
                if lst[i] + 1 == lst[i+1] and lst[i+1] + 1 == lst[i+2]:
                    triples.append((lst[i], lst[i+1], lst[i+2]))
        return triples
    def filter_triples(lst):
        l = lst
        ml = []
        for t in get_trilples(lst):
            ml.append(t[1])
            l.remove(t[0])
            l.remove(t[2])
        return l, ml
    out = im
    h, w, *_ = out.shape
    out = cv.inRange(out, grid_color, grid_color)
    r = []
    for ax, mx in zip([0, 1], [h, w]):
        s = np.sum(out/255, axis=ax)
        lines = list(np.where(np.abs(s - mx) <= threshold)[0])
        l, ml = filter_triples(lines)
        r.append((l, ml))
    nodes = list(itertools.product(r[0][0], r[1][0]))
    
    return list(itertools.product(r[0][0], r[1][0])), r[0][1], r[1][1]


with timer_ms() as elapsed:
    nodes, mvl, mhl = get_grid(im)
    print(f'elapsed: {elapsed()}')

for n in nodes:

    if n[1] in mhl and n[0] in mvl:
        cv.circle(im, n, 5, (255, 0, 0), 2)
    elif n[1] in mhl:
        cv.rectangle(im, (n[0], n[1] - 3), (n[0], n[1] + 3), (255, 0, 0), 2)
    elif n[0] in mvl:
        cv.rectangle(im, (n[0] - 3, n[1]), (n[0] + 3, n[1]), (255, 0, 0), 2)
    else:
        cv.circle(im, n, 2, (255, 255, 0), 1)
p = nodes[0]
ds = set()
for n in nodes[1:]:
    # print(p, n, f'p: {p[0]}, {p[1]}, n: {n[0]}, {n[1]}')
    if n[0] == p[0]:
        d1 = n[1] - p[1]
        ds.add(d1)
    else:
        d1 = n[0] - p[0]
        ds.add(d1)
    p = n
print(ds)

img = Image.fromarray(im.astype('uint8'))
display(img)


x = '''
kernel_size = 99
kernel_v = np.zeros((kernel_size, kernel_size)) 
kernel_h = np.copy(kernel_v) 
kernel_v[:, int((kernel_size - 1)/2)] = np.ones(kernel_size) 
kernel_h[int((kernel_size - 1)/2), :] = np.ones(kernel_size) 
kernel_v /= kernel_size 
kernel_h /= kernel_size 
outv = cv2.filter2D(out, -1, kernel_v) 
outv = cv.inRange(outv, 200, 255)
outh = cv2.filter2D(out, -1, kernel_h) 
outh = cv.inRange(outh, 200, 255)
out = outh
out = cv.bitwise_and(outv, outh)

# out = erode(out, 1, cv.MORPH_RECT)
# out = erode(out, 1, cv.MORPH_ELLIPSE)
  
# Save the outputs. 
# cv2.imwrite('car_vertical.jpg', vertical_mb) 
# cv2.imwrite('car_horizontal.jpg', horizonal_mb) 


#out = cv.bilateralFilter(out, 9, 75, 75)
# out = erode(out, 1, cv.MORPH_ELLIPSE)
# ds2 = 1
# el_ellipse = cv.getStructuringElement(cv.MORPH_ELLIPSE, (2 * ds1 + 1, 2 * ds1 + 1), (ds1, ds1))
# v = cv.erode(v, el_ellipse)
y, x = np.where(out != 0)
c = list(sorted(zip(x, y)))
# print(c)
# logging.info(f'{str(im.shape)}, {str(im.data.format)}')

# c = [p for p in zip(x, y)]
    #cv.circle(img_rgb, pt,7, (255,0,0), 3)
    # cv.circle(v, p, 7, 255, 2)
cv.imwrite('../tmp/out.bmp', out)

# out = erode(out, 1, cv.MORPH_RECT)
# cv.imwrite('../tmp/out1.bmp', out)

wmax = 128
win = out[0:wmax, 0:wmax]
# cv.imwrite('../tmp/win.bmp', win)
# win = cv.imread('../tmp/win1.bmp', cv.COLOR_BGR2GRAY)
x, y = np.where(win != 0)
# print(x[0])

nodes = np.array(list(zip(x, y)))

def chk(img, p):
    v = np.array([[0, +1], [0, -1]])
    h = np.array([[+1, 0], [-1, 0]])
    # e = np.array([[-1, -1], [0, -1], [1, -1], [-1, 0], [1, 0], [-1, 1], [0, 1], [1, 1]])
    # ep =list(map(lambda x: img[*(x + p)] == 0, e))
    p = np.array(p)
    vp = list(map(lambda x: img[*(x + p)] == 255, v))
    nvp = list(map(lambda x: img[*(x + p)] == 0, v))
    hp = list(map(lambda x: img[*(x + p)] == 255, h))
    nhp = list(map(lambda x: img[*(x + p)] == 0, h))
    p, q, r, s = all(vp), all(hp), img[*p], True
    if all(vp) and all(hp) and r:
        return 'big'
    if all(vp) and all(nhp) and r:
        return 'vert'
    if all(hp) and all(nvp) and r:
        return 'horz'
    if all(nvp) and all(nhp) and r:
        return 'small'
    return 'none'

qqqq = [n for n in nodes if chk(win, n) != 'none']
#qqqq = list(filter(lambda x: chk(win, x) != 'none', nodes))
#qqqq = list(map(lambda x: (chk(win, x), x), nodes))
# print(len(qqqq), len(nodes))

# p = np.array((425, 400))

ii = np.zeros_like(win)

# for t, n in qqqq:
#     if t == 'big':
#         ii[*n] = 255
#     elif t == 'vert' or t == 'horz':
#         ii[*n] = 120
#     elif t == 'small':
#         ii[*n] = 58


qqqq1 = list(sorted(qqqq, key=lambda x: 10000*x[0] + x[1]))

for n in qqqq1:
    ii[*n] = 255

cv.imwrite('../tmp/ii.bmp', ii)



n0, n1 = qqqq1[0], qqqq1[1]


if n0[0] == n1[0]:
    d = n1[1] - n0[1]
else:
    d = n1[0] - n0[0]

print(f'grid size: {d}')

p = qqqq1[0]
ds = set()
for n in qqqq1[1:]:
    # print(p, n, f'p: {p[0]}, {p[1]}, n: {n[0]}, {n[1]}')
    if n[0] == p[0]:
        d1 = n[1] - p[1]
        ds.add(d1)
        # print(d1)
        assert abs(d1 - d) < 2
    else:
        d1 = n[0] - p[0]
        ds.add(d1)
        assert abs(d1 - d) < 2
    p = n

print(ds)
# print(qqqq1)
# ii[*qqqq] = 255


img = Image.fromarray(out.astype('uint8'))
# scaled = img.resize(list(map(lambda x: int(0.5 * x), img.size)))
# display(img)
'''

In [ ]:
[1,2,3][1:]

In [ ]:
from typing import List, Tuple
import cv2 as cv
import cv2
import numpy as np
from PIL import Image
from IPython.display import display
import sys

def erode(img: np.ndarray, sz: int, shape):
    el = cv.getStructuringElement(shape, (2 * sz + 1, 2 * sz + 1), (sz, sz))
    v = cv.erode(img, el)
    return v

def dilate(img: np.ndarray, sz: int, shape):
    el = cv.getStructuringElement(shape, (2 * sz + 1, 2 * sz + 1), (sz, sz))
    v = cv.dilate(img, el)
    return v

#del out, outv, outh

im = cv.imread('../tmp/test_get_non_ui_screen_non_ui_img.bmp')
out = im
black = (0, 0, 0)
out = cv.inRange(out, black, black)
kernel_size = 99
kernel_v = np.zeros((kernel_size, kernel_size)) 
kernel_h = np.copy(kernel_v) 
kernel_v[:, int((kernel_size - 1)/2)] = np.ones(kernel_size) 
kernel_h[int((kernel_size - 1)/2), :] = np.ones(kernel_size) 
kernel_v /= kernel_size 
kernel_h /= kernel_size 
outv = cv2.filter2D(out, -1, kernel_v) 
outv = cv.inRange(outv, 200, 255)
outh = cv2.filter2D(out, -1, kernel_h) 
outh = cv.inRange(outh, 200, 255)
# out = outh
out = cv.bitwise_and(outv, outh)

img = Image.fromarray(out.astype('uint8'))

display(img)


In [ ]:
def d(a, b, c):
    print('/////////')
    print(a, b, c)
    print('---')
    return 123

x = '''
d(1, '2', 123 + 1)

x = 999_999
if x < 1000_000:
    print("aaaaaaa, я кусака!!!!!")

for x in [4, 200, 101, 1, 1000_0000]:

    if x < 4:
        break

    print(x,)


def 0000000000000000000000000000000000000000000000000000000000000000000000000000inc(s, l = 10):
    if len(s) < l:
        inc(s + '.')
'''
i = -1
j = 1
f'{j:+03}'


import itertools
a = list(range(7,14))
b = list(range(3,6))
l = list(itertools.product(a, b))

import random
random.randint(1, 10)
random.shuffle(l)
l[:5]

x = next(filter(lambda x: type(x) == str, [0, True, 1]), None)
print(repr(x))